In [17]:
#standard class library imports
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pymongo
from collections import OrderedDict
import numpy as np
import requests
import json
from config import api_key
from pprint import pprint


In [18]:
#named entity recognition
import spacy
from spacy import displacy
NER = spacy.load("en_core_web_sm")


In [19]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# Define database and collection
db = client.troublesfinal_db
collection = db.items

In [20]:
### SCRIPT FOR HTML PAGE SCRAPE ###

def text_cleanup(dfrow):
    death = str(scrapedf[dfrow])
    deathsplitspace = death.split()


    #dateofdeath string established and parsed
    deathindex = deathsplitspace.index(year)
    dateofdeath = []
    for deathsplitspaceindex in range(deathindex+1):
        dateofdeath.append(deathsplitspace[deathsplitspaceindex])
    #???:how to convert 'dateofdeath' from string into datetime
    ##print(dateofdeath)

    #namestring established
    indices = [i for i, s in enumerate(deathsplitspace) if '(' in s]
    namestring = []
    lastname = []
    firstname = []
    namerange = range(deathsplitspaceindex+1,indices[0])
    for namecomps in range(deathsplitspaceindex+1,indices[0]):
        namestring.append(deathsplitspace[namecomps])

    #namestring parsing    
    for namecomp in namestring:
        if namecomp.endswith(",") == True:
            lastnameendvalue = namecomp
            lastnameendindex = namestring.index(lastnameendvalue)
    for lastnameind in range(lastnameendindex+1):
        lastname.append(namestring[lastnameind])

    if lastnameendindex+1 == len(namestring)-1:
        firstname.append(namestring[lastnameendindex+1])
    else:
        for firstnameind in range(lastnameendindex+1,len(namestring)-1):
            firstname.append(namestring[firstnameind])
    lastname = [s.replace(",", "") for s in lastname]
    ##print(lastname)
    ##print(firstname)

    #affiliation established
        #excluding for now

    #'killed by' established
    killedbystartindices = [i for i, s in enumerate(deathsplitspace) if 'Killed' in s]
    killedbyendindices = [i for i, s in enumerate(deathsplitspace) if ')' in s]
    killedbystartindice = killedbystartindices[0]
    killedbyendindice = killedbyendindices[2]
    killedby = []

    for killedbycomps in range(killedbystartindice+2,killedbyendindice):
        killedby.append(deathsplitspace[killedbycomps])

    ##print(killedby)

    #details established
    details = []
    for detailcomp in range(killedbyendindice, len(deathsplitspace)):
        details.append(deathsplitspace[detailcomp])

    ##print(details)
    
    #spacy function to return entity info/create Google Places search string
    listforgoogleplaces = []
    def show_ents(doc):
        if doc.ents:  
            for ent in doc.ents:
                listforgoogleplaces.append(str(ent.text))
    #passing details through NER
    parsed_details = NER(' '.join(details))

    for NamedEntity in parsed_details.ents:
        show_ents(NamedEntity)
        
    #clean list of NER to remove dates and lower case NER's
    listofUpperCaseforgoogleplaces = []
    for nercomps in listforgoogleplaces:
        if nercomps[0].isupper():
            listofUpperCaseforgoogleplaces.append(nercomps)
            

    stringforgoogleplaces = ' '.join(listofUpperCaseforgoogleplaces)
    
    
    #PASS 'stringforgoogleplaces' INTO GOOGLE PLACES API
    
    
    #target_city = 'divis street and dover street'
    key = api_key

    target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={stringforgoogleplaces}&key={key}"

    # run a request using our params dictionary
    response = requests.get(target_url)

    # convert response to json
    places = response.json()

    # Print the json (pretty printed)
    ##print(json.dumps(places, indent=4, sort_keys=True))


    # Extract latitude and longitude
    try:
        lat = places["results"][0]["geometry"]["location"]["lat"]
        lng = places["results"][0]["geometry"]["location"]["lng"]
    except:
        lat = "Not Found"
        lng = "Not Found"
    # Print the latitude and longitude
    ##print('''
        #City: {0}
        #Latitude: {1}
        #Longitude: {2}
        #'''.format(target_city, lat, lng))
    
    post = {
                'Date of Death': dateofdeath,
                'First Name': firstname,
                'Last Name': lastname,
                'Killed by:': killedby,
                'Latitude':lat,
                'Longitude':lng
            }

    collection.insert_one(post)
   



In [21]:
### WRITE TO MONGODB ###

#define/populate list of year pages of deaths
yearsofdeaths = []
for i in range(1969,2002):
    yearsofdeaths.append(str(i))
    
#loop through all year pages and call scrape function/upload to MONGODB    
for year in yearsofdeaths:
    combinedurl = 'https://cain.ulster.ac.uk/sutton/chron/'+str(year)+'.html'
    tables = pd.read_html(combinedurl)
    df = tables[1]
    scrapedf = df.iloc[:,1]
    for row in range (len(scrapedf)):
        text_cleanup(row)
    print(year + ' has been Uploaded!')

1969 has been Uploaded!
1970 has been Uploaded!
1971 has been Uploaded!
1972 has been Uploaded!
1973 has been Uploaded!
1974 has been Uploaded!
1975 has been Uploaded!
1976 has been Uploaded!
1977 has been Uploaded!
1978 has been Uploaded!
1979 has been Uploaded!
1980 has been Uploaded!
1981 has been Uploaded!
1982 has been Uploaded!
1983 has been Uploaded!
1984 has been Uploaded!
1985 has been Uploaded!
1986 has been Uploaded!
1987 has been Uploaded!
1988 has been Uploaded!
1989 has been Uploaded!
1990 has been Uploaded!
1991 has been Uploaded!
1992 has been Uploaded!
1993 has been Uploaded!
1994 has been Uploaded!
1995 has been Uploaded!
1996 has been Uploaded!
1997 has been Uploaded!
1998 has been Uploaded!
1999 has been Uploaded!
2000 has been Uploaded!
2001 has been Uploaded!
